In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai

openai.api_key = "<YOUR_OPENAI_KEY>"

## load

First we have to scrape the text.

We use requests to get the text, and Beautiful Soup to gather the content.

For my blog, my posts the `<a>` tags so I use find_all() to filter them out.

Your website might have a different HTML structure, so you might need to spend some time for BeautifulSoup to get the right thing. Pro tip: use ChatGPT to help you here.

Then, a list comprehension to gather all the links that direct to my posts.


In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
import re

url = "https://bneo.xyz/posts"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

article_links = soup.find_all("a", href=re.compile("posts/"))  # find all a tags
relative_urls = [link["href"] for link in article_links if link["href"].startswith("/")]

urls = [f"https://bneo.xyz{relative_url}" for relative_url in relative_urls]
urls

['https://bneo.xyz/posts/1yearreview',
 'https://bneo.xyz/posts/michaelhan',
 'https://bneo.xyz/posts/selfjudgement',
 'https://bneo.xyz/posts/community',
 'https://bneo.xyz/posts/naval',
 'https://bneo.xyz/posts/goodconvos',
 'https://bneo.xyz/posts/artofgather',
 'https://bneo.xyz/posts/5qsclimate',
 'https://bneo.xyz/posts/climate',
 'https://bneo.xyz/posts/edwardmunch',
 'https://bneo.xyz/posts/proustquestions',
 'https://bneo.xyz/posts/webdev',
 'https://bneo.xyz/posts/joandidion',
 'https://bneo.xyz/posts/workdoc',
 'https://bneo.xyz/posts/donelist',
 'https://bneo.xyz/posts/pmarca-career',
 'https://bneo.xyz/posts/310324',
 'https://bneo.xyz/posts/typeIIfun',
 'https://bneo.xyz/posts/lowfocus',
 'https://bneo.xyz/posts/beautyofbooks',
 'https://bneo.xyz/posts/4hours',
 'https://bneo.xyz/posts/lookart',
 'https://bneo.xyz/posts/o1',
 'https://bneo.xyz/posts/truefriendship',
 'https://bneo.xyz/posts/origincreation',
 'https://bneo.xyz/posts/solarstorms',
 'https://bneo.xyz/posts/s

In [5]:
len(urls)

367

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("main")),  # only elements in main tag
)

docs = loader.load()

print(f"{len(docs)} documents loaded")

367 documents loaded


In [7]:
print(docs[0].page_content[:500])

1 Year of BloggingClaude Monet, Shadows on the Sea. The Cliffs at Pourville, 1882
Exactly a year ago I started blogging everyday.
These blogs are mostly good ideas that I discover online that I want to compartmentalize and steal for myself, and collect them here in my own corner of the internet so I can revisit them from time to time.
As for my reflective and introspective writings, they're an attempt to document my life and for making sense of my thoughts.

“Write hard and clear about what hurt


In [8]:
print(sum(len(doc.page_content) for doc in docs))

924688


## split


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(docs)

In [10]:
len(splits)

1376

In [11]:
from pprint import pprint

pprint(splits[:5])

[Document(page_content="1 Year of BloggingClaude Monet, Shadows on the Sea. The Cliffs at Pourville, 1882\nExactly a year ago I started blogging everyday.\nThese blogs are mostly good ideas that I discover online that I want to compartmentalize and steal for myself, and collect them here in my own corner of the internet so I can revisit them from time to time.\nAs for my reflective and introspective writings, they're an attempt to document my life and for making sense of my thoughts.\n\n“Write hard and clear about what hurts. ― Ernest Hemingway\n\nReading back my journal entries in this blog is almost like time travelling. I'm visiting different versions of my past selves, unpacking and resurfacing all their thoughts, ambitions, sufferings, fears, anxiety, and emotions.\nSome I've conquered, some I still harbor, but it's comforting to see growth and change happen. I'm optimistic for April 16, 2025: the kind of person I'll become and the people that will be in my life in SF.", metadata=

In [12]:
len(splits[0].page_content)

956

In [13]:
splits[20].metadata

{'source': 'https://bneo.xyz/posts/goodconvos', 'start_index': 1482}

## store


In [14]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

## retrieve


In [15]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [16]:
retrieved_docs = retriever.invoke("What is love?")

In [17]:
len(retrieved_docs)

6

In [18]:
print(retrieved_docs[3].page_content)

Your doubt may become a good quality if you train it. It must become knowing, it must become critical. Ask it, whenever it wants to spoil something for you, why something is ugly, demand proofs from it, test it, and you will find it perplexed and embarrassed perhaps, or perhaps rebellious. But don’t give in, insist on arguments and act this way, watchful and consistent, every single time, and the day will arrive when from a destroyer it will become one of your best workers — perhaps the cleverest of all that are building at your life.

Love

To love is good too, for love is difficult. For one person to care for another, that is perhaps the most difficult thing required of us, the utmost and final test, the work for which all other work is but a preparation. With our whole being, with all the strength we have gathered, we must learn to love. This learning is ever a committed and enduring process.


## generate


In [24]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Write with simple language in Paul Graham style. Write at least least 5 sentences.

{context}

Question: {question}

Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [25]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
rag_chain.invoke("What is love?")

"Love, it seems, is a complex and multifaceted concept that cannot be easily defined by a single statement. It's an endeavor that challenges us, demands our utmost effort, and requires a continuous process of learning and growth. Love is not merely about the merging of two beings into one but rather about the individual growth that occurs in response to another person. It calls for the development of one's own world, respecting and greeting the solitude of the other, thereby creating a bond that is both protective and respectful of each other's boundaries.\n\nLove also encompasses the willingness to endure suffering and pain, recognizing that these experiences are integral to the human condition and can lead to deeper understanding and knowledge. It's about embracing doubt and transforming it into a tool for critical thinking and self-improvement. Love involves patience, not only with others but with oneself, and the courage to live with unanswered questions, finding solace in the jour

In [27]:
for chunk in rag_chain.stream("What is love?"):
    print(chunk, end="", flush=True)

Love, it seems, is a complex and multifaceted concept that defies simple definition. It's not merely a feeling of affection or attraction towards someone. Rather, it's a profound, challenging, and transformative force that demands the best of us. Love calls for individuals to grow, to differentiate, and to become worlds unto themselves in response to another. It's about two solitudes that protect, border, and greet each other, not losing themselves in the process but enriching each other's lives.

Love involves suffering and happiness, pain and joy. It's an incurable condition that brings about permanent suffering, yet it's also the highest calling that requires us to care deeply for another person. This caring is perhaps the most difficult task asked of us, a task for which all other tasks are but preparation. Love demands patience, learning, and the willingness to live with questions that cannot yet be answered.

In essence, love is about growth, both personal and mutual. It's about 

### adding sources


In [28]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)


rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is Love?")

{'context': [Document(page_content='Be Happy in Love', metadata={'source': 'https://bneo.xyz/posts/proust', 'start_index': 9221}),
  Document(page_content='To love is not about merging. It is a noble calling for the individual to ripen, to differentiate, to become a world in oneself in response to another. It is a great, immodest call that singles out a person and summons them beyond all boundaries. Only in this sense may we use the love that has been given us. This is humanity’s task, for which we are still barely ready.\n\n\nThis more human love (endlessly considerate and light and good and clear, consummated by holding close and letting go) will resemble that love that we so arduously prepare — the love that consists of two solitudes that protect, border, and greet each other.\n#book2/9/2024Next:Writing Book SummariesPrevious:YouTube Archive', metadata={'source': 'https://bneo.xyz/posts/rilkeletters', 'start_index': 8611}),
  Document(page_content='Your doubt may become a good quali

In [29]:
def format_to_markdown(data):
    markdown_output = (
        f"Question:{data['question']}\n\nAnswer:\n{data['answer']}\n\nSources:\n\n"
    )
    for i, doc in enumerate(data["context"], start=1):
        page_content = doc.page_content.split("\n")[
            0
        ]  # Get the first line of the content for brevity
        source_link = doc.metadata["source"]
        markdown_output += f"[[{i}]({source_link})] {page_content}\n\n"
    return markdown_output

In [30]:
from IPython.display import display, Markdown

res = rag_chain_with_source.invoke("What is Love?")
display(Markdown(format_to_markdown(res)))

Question:What is Love?

Answer:
Love, as distilled from the various contexts provided, is a complex, multifaceted experience that defies simple definition. It's not merely a feeling of affection or attraction towards another person; it's a profound, challenging, and transformative process. Love calls for individuals to grow, to embrace their solitude while reaching out to another, protecting and respecting each other's boundaries. It's about learning to care deeply for another person, which is perhaps one of the most difficult tasks we are faced with in life. Love involves suffering and risk, but it's also seen as a path to knowledge and self-discovery. Through the pain and challenges of love, we can learn about our emotional dependencies and grow from them. Love is worth the risk because, through this arduous journey, we find opportunities for profound personal growth and understanding.

Sources:

[[1](https://bneo.xyz/posts/proust)] Be Happy in Love

[[2](https://bneo.xyz/posts/rilkeletters)] To love is not about merging. It is a noble calling for the individual to ripen, to differentiate, to become a world in oneself in response to another. It is a great, immodest call that singles out a person and summons them beyond all boundaries. Only in this sense may we use the love that has been given us. This is humanity’s task, for which we are still barely ready.

[[3](https://bneo.xyz/posts/rilkeletters)] Your doubt may become a good quality if you train it. It must become knowing, it must become critical. Ask it, whenever it wants to spoil something for you, why something is ugly, demand proofs from it, test it, and you will find it perplexed and embarrassed perhaps, or perhaps rebellious. But don’t give in, insist on arguments and act this way, watchful and consistent, every single time, and the day will arrive when from a destroyer it will become one of your best workers — perhaps the cleverest of all that are building at your life.

[[4](https://bneo.xyz/posts/proust)] Suffer Successfully

[[5](https://bneo.xyz/posts/videos)] YouTube ArchiveThe Great Library of Alexandria, O. Von Corven, 19th century

[[6](https://bneo.xyz/posts/videos)] Artists



In [31]:
def ask(q):
    res = rag_chain_with_source.invoke(q)
    return display(Markdown(format_to_markdown(res)))

In [32]:
ask("What is the meaning of life?")

Question:What is the meaning of life?

Answer:
The meaning of life isn't a one-size-fits-all answer, like a best move in chess that doesn't exist. It's more about the unique journey each person takes, the specific mission or vocation they find for themselves. Life isn't asking us to solve it like a riddle but is instead posing questions to us daily, demanding our response through the way we live and the responsibilities we take on.

Life is an endless process of unfolding, a dialogue between our potentialities and the situations we find ourselves in. These potentialities, ranging from intellectual gifts to the capacity for love and aspiration, don't have an expiration date but grow through the challenges life throws at us. It's in facing these challenges that we find meaning, especially in suffering, which is an inseparable part of life. Suffering and the way we respond to it can bring out values and strengths we never knew we had.

Existentialists argue that to live is to suffer, but within that suffering, we find the opportunity to give life meaning. Nietzsche famously said, "He who has a why to live can bear almost any how." This suggests that finding a purpose or a 'why' in life can help us endure the hardships. The essence of existence, then, is not about seeking a universal meaning of life but about taking responsibility for finding our own answers to the problems life sets before us, embracing our suffering as part of our unique path, and living in a way that is true to ourselves and our potentialities. Life is about the journey, the challenges, and how we choose to respond to them.

Sources:

[[1](https://bneo.xyz/posts/meaning)] Meaning of life

[[2](https://bneo.xyz/posts/renewal)] But life isn't a mountain that has a summit, Nor is it -- as some suppose -- a riddle that has an answer. Nor a game that has a final score.

[[3](https://bneo.xyz/posts/meaning)] central theme of existentialism: to life is to suffer, to survive is to find meaning in the suffering.

[[4](https://bneo.xyz/posts/meaning)] Spinoza: "Emotion, which is suffering, ceases to be suffering as soon as we form a clear and precise picture of it.

[[5](https://bneo.xyz/posts/happy-life)] "Don't ever confuse the two, your life and your work"

[[6](https://bneo.xyz/posts/meaning)] A man who becomes conscious of the responsibility he bears towards a human being who affectionately waits for him or to an unfinished work, will never be able to throw away his life.



In [33]:
ask("What is a community?")

Question:What is a community?

Answer:
A community is a tapestry of individuals, each with their own stories, backgrounds, and perspectives, woven together by shared spaces, experiences, or goals. It's not just a group of people living in the same place, but a network of relationships built on mutual support, understanding, and sometimes, the simple act of playing freely on the streets. Communities thrive on the Ubuntu philosophy: "I am because you are." This means recognizing that our own well-being is deeply connected to the well-being of others around us. In a community, members engage with each other in unscripted, unmonitored ways, learning to navigate the complexities of human relationships through direct interaction. This process helps to rebuild the social muscles weakened by our reliance on screen-based, friction-free solutions to meet our needs. In essence, a community is a collective where individuals come together to share their joys, sorrows, and life's mundane moments, enriching each other's lives through the power of connection and shared humanity.

Sources:

[[1](https://bneo.xyz/posts/play-streets)] We're so burned out from our screen-based, data-heavy lives that we no longer have time or energy for the small, mundane, place-based friendships that has nourished and sustained us for centuries.

[[2](https://bneo.xyz/posts/play-streets)] You get it all served in uncomplicated, lack of friction, and you no longer know how to deal with people.

[[3](https://bneo.xyz/posts/braces)] the idea Ubuntu philosophy: I am because you are; I cannot be content and live a fulfilled life if the people around me are suffering, are struggling. it's very much a collective social ideology, a stark contrast with the wellness rooted in individualism today.

[[4](https://bneo.xyz/posts/burnoutsociety)] Burnout society

[[5](https://bneo.xyz/posts/play-streets)] Playing Freely on the StreetsWhen you grew up, did you play freely on the street?

[[6](https://bneo.xyz/posts/party)] Everyone is thinking about it, but nobody knows what to do.



In [34]:
ask("What is Boston like?")

Question:What is Boston like?

Answer:
Boston is a city of contrasts and history, a place where the old and the new coexist in a fascinating dance. It's a city where navigating can be tricky, thanks to its unique road layout and the occasional GPS glitch, especially amidst the towering buildings. Yet, it's also a city that offers moments of beauty, like a sunset at Rowes Wharf that can make you feel as if you're inside a painting. Walking through Boston, especially after spending time in a quieter place like Ames, feels like a reawakening, a rush of stimuli from every direction, reminding you of the vibrancy of urban life.

The city is not just about its streets and buildings; it's about experiences. From the chewy yet satisfying lobster rolls priced at $40, which speak to the city's culinary delights, to the cozy and inviting atmosphere of the Beacon Hill Books & Cafe, Boston offers a variety of experiences that cater to different tastes and interests. The city's rich history is palpable in its historical sites like the Old South Meeting House and the Massachusetts State House, inviting those with a penchant for the past to explore and learn.

Moreover, Boston's cultural scene is vibrant, with institutions like the Isabella Stewart Gardner Museum offering a glimpse into the aesthetic and artistic tastes of one of the city's iconic figures. The museum, with its beautiful garden and three floors of art, tells a personal story that enriches the visitor's experience of the city.

Boston is also a place for discovery and exploration, with areas like Newbury Street offering shopping and dining experiences that can cap off a day of museum visits or historical tours. The city's mix of educational institutions, historical depth, cultural richness, and urban energy makes it a complex but fascinating place to visit, where every corner holds a story and every experience adds a layer to one's understanding of this historic city.

Sources:

[[1](https://bneo.xyz/posts/boston-d1)] It was $40 for a good amount of lobster meat and bread. I found it to be chewy at first, but grew to like it.

[[2](https://bneo.xyz/posts/boston-d3)] Boston Day 3: Isabella & MFAIsabella Stewart Gardner Museum

[[3](https://bneo.xyz/posts/boston-d3)] The bookstore was so nice.

[[4](https://bneo.xyz/posts/boston-d2)] Great Dome (grab a picture here)

[[5](https://bneo.xyz/posts/boston-d1)] Boston Day 1: Quincy MarketQuincy market

[[6](https://bneo.xyz/posts/boston-d1)] Arriving in Boston, we were full of excitement and positive anticipation.



In [35]:
ask("What is California like?")

Question:What is California like?

Answer:
California, through the eyes of someone who has lived and breathed its air, is a place of endless possibilities. It's a land where the tech industry thrives, where innovation and creativity meet at every corner, especially in the Bay Area and Silicon Valley. The weather is good, contributing to an atmosphere that's ripe for adventure and exploration. People come from all over with dreams and passions, making it easy to find others who share similar interests and drives. The culture is diverse, and there's always something happening—be it hackathons, meetups, or just exploring the vibrant cities and beautiful landscapes. California seems to embody a spirit of excitement and drive, making every weekend full of potential new experiences. It's a place that can make you feel at home, even if you're living out of Airbnbs, because of the connections you make and the shared energy of striving for something more.

Sources:

[[1](https://bneo.xyz/posts/making-friends)] Friendships in CaliforniaI read this article about making friends as an adult, and it made me reflect about my time in Cali.

[[2](https://bneo.xyz/posts/isu)] Going to California was a breath of fresh air. I got to experience the bay area. The Silicon Valley. It was what I dreamt about when I started college. It was also my first job ever in my life. I got to work at the factory where Tesla cars are made and see Elon Musk in person. I made a lot of friends, saw a lot of places, ate good (limited) food. I grew a lot that summer.

[[3](https://bneo.xyz/posts/isu)] After a really fun tokyo trip in winter, and the obligatory return to Malaysia, I came back to the cold and unforgiving weather, still trying to get an internship.

[[4](https://bneo.xyz/posts/22)] Leaving California again was hard. My old friends came back full time, and I felt like I was delaying my life. I also had only one more class to graduate, and I was contemplating a lot about coming back to Iowa. But now I'm glad I did. I was really burnt out in California, and never took the time to rest.

[[5](https://bneo.xyz/posts/malaysia)] To be this person, this voice tells me I have to be in the US, specifically the Bay Area. San Francisco. After two summers in the bay, I felt really at home, even though I moved between 5 different Airbnbs, I found my people. I met people who thought about the same things I did, who shared the same passion and drive for things, who I could relate to at a deeper level. I felt like I've known them forever and I wish I met them sooner. My days were filled with excitement and drive, my weekends were full of possibilities. I made plans every week. I talked to strangers at hackathons and talks. My most vivid memory was walking the streets of SF alone for an hour searching for a ramen after spending 24 hours without sleep for an AI hackathon, talking to startup founders and MIT students, eating bags of Fritos Corn Chips, and winning a category prize.

[[6](https://bneo.xyz/posts/backhome)] That said, I miss a lot of things about the US, and everything I see and hear in Malaysia just makes me miss them more. I'm leaving what to me, is a better life in all aspects. A higher paying job, my friends, good weather, old bookstores, California and Boston and Seattle, the tech industry, the hackathons, the cold, the accents, the culture, the atmosphere, my comfy room with 0 mess. I miss all this. a lot. Perhaps I should heed the advice of Proust, to see the beauty in everyday objects and things, take my time on what seems boring at first, to not let snobbery get in the way of appreciation of my culture, my home, and my people.



In [52]:
ask("Name me all the places in California you can find in bullet points")

Question:Name me all the places in California you can find in bullet points

Answer:
- SF (San Francisco)
- Outer Sunset on Irving Street
- Golden Gate Park
- De Young Museum
- California hiking spots
- Tesla factory (Silicon Valley)
- Fort Baker
- Golden Gate View Point
- Good Luck Dim Sum on Clement St
- Portsmouth Square
- St Mary's Square
- Union Square

Sources:

[[1](https://bneo.xyz/posts/deyoung)] Arts market and MuseumToday was an eventful day.

[[2](https://bneo.xyz/posts/making-friends)] Friendships in CaliforniaI read this article about making friends as an adult, and it made me reflect about my time in Cali.

[[3](https://bneo.xyz/posts/golden-gate)] SF Trip w ParentsDay #2 of bringing my parents around to touristy spots in California.

[[4](https://bneo.xyz/posts/malaysia)] To be this person, this voice tells me I have to be in the US, specifically the Bay Area. San Francisco. After two summers in the bay, I felt really at home, even though I moved between 5 different Airbnbs, I found my people. I met people who thought about the same things I did, who shared the same passion and drive for things, who I could relate to at a deeper level. I felt like I've known them forever and I wish I met them sooner. My days were filled with excitement and drive, my weekends were full of possibilities. I made plans every week. I talked to strangers at hackathons and talks. My most vivid memory was walking the streets of SF alone for an hour searching for a ramen after spending 24 hours without sleep for an AI hackathon, talking to startup founders and MIT students, eating bags of Fritos Corn Chips, and winning a category prize.

[[5](https://bneo.xyz/posts/chinatown)] The plan was to go Portsmouth Square, and find a place to eat.

[[6](https://bneo.xyz/posts/isu)] Going to California was a breath of fresh air. I got to experience the bay area. The Silicon Valley. It was what I dreamt about when I started college. It was also my first job ever in my life. I got to work at the factory where Tesla cars are made and see Elon Musk in person. I made a lot of friends, saw a lot of places, ate good (limited) food. I grew a lot that summer.



In [36]:
ask("what are some useful macos apps")

Question:what are some useful macos apps

Answer:
If you're looking for useful MacOS apps, you're in luck. There's a treasure trove of tools out there designed to enhance productivity, streamline your workflow, and just make life a bit easier. For starters, AltTab brings the Windows-style app switching to MacOS, making it easier to navigate between applications. Amie combines your calendar, to-dos, and integrations in one place, simplifying how you organize your day. For those concerned about their digital footprint, App Cleaner helps you uninstall apps completely, leaving no leftover files behind.

Arc claims the title of the best browser on Earth, a bold statement but one that many users agree with due to its speed and efficiency. BatFi is another gem, especially for those looking to optimize their Mac's battery life by stopping the charge at 80%. And for security-conscious users, Bitwarden stands out as a robust password manager. These are just a few examples, but they highlight the diversity and utility of apps available for MacOS users. Whether you're looking to enhance productivity, manage your digital life, or simply enjoy a better browsing experience, there's likely an app out there for you.

Sources:

[[1](https://bneo.xyz/posts/apps)] Useful MacOS AppsThis is a living post of useful apps/tools I use on MacOS.

[[2](https://bneo.xyz/posts/focus)] Tools for Keeping FocusSome things I do to help keep focus on my Macbook and iPhone, inspired by this LessWrong post by Ben Kuhn

[[3](https://bneo.xyz/posts/ios)] Some IOS Apps I Likea few ios apps on my i13 mini

[[4](https://bneo.xyz/posts/swift)] ios twitter

[[5](https://bneo.xyz/posts/focus)] browser

[[6](https://bneo.xyz/posts/admire)] Some more inspirations off the top of my head as I write this. this list is very much non-exhaustive.



In [37]:
ask("Advice for 20 year olds")

Question:Advice for 20 year olds

Answer:
In your 20s, think of life as a grand experiment. It's not about climbing the highest ladder as fast as you can. It's about learning who you are, what you love, and how you fit into the world. This decade is your golden time to collect experiences and people that shape you. Work in different places, even if they seem off the beaten path, like another country or a farm. These experiences grow you in ways a corporate job never will.

Fail a few times. Pick the wrong job, take on risky projects. Through failure, you learn about your strengths and weaknesses. Make a list of what you love and hate, what you're good at, and what you're not. The intersection of what you love and excel at will guide you to roles where you'll be both successful and happy.

Remember, happiness and frustration are two sides of the same coin. Manage your expectations about life and work hard. The good life requires effort, both in your career and personal relationships. Rethink your desires. If wealth is your goal, understand why. The best reason to seek wealth is to have the freedom to do what you love, not to escape work.

The world has changed. You won't likely stay with one company your entire career. Your first job's purpose is to lead to a better one. Balance your time between planning for the future and enjoying the present. Develop your people skills; they're crucial in every aspect of life.

Lastly, bias towards action. Your 20s are for doing, experimenting, and stepping out of your comfort zone. This is how you grow, learn, and find your path. Remember, "the good life is a life inspired by love and guided by knowledge." Keep these principles in mind, and you'll navigate your 20s with purpose and joy.

Sources:

[[1](https://bneo.xyz/posts/career-advice)] Career Advice for 20sMolly Graham shares 10 things she wish she knew at the beginning of her 20s.

[[2](https://bneo.xyz/posts/gradadvice)] Advice for New College GradsSome advice for new college grads by Chris from 2006.

[[3](https://bneo.xyz/posts/8artist)] We're so immersed in the bullsh*t of our own activities to know even how to give anyone advice

[[4](https://bneo.xyz/posts/career-advice)] More career advice:

[[5](https://bneo.xyz/posts/action)] Bias Towards ActionYour 20s is the time do do random and unexpected things.

[[6](https://bneo.xyz/posts/gradadvice)] "the good life is a life inspired by love and guided by knowledge" - Bertrand Russell



In [38]:
ask("how to deal with anxiety?")

Question:how to deal with anxiety?

Answer:
Dealing with anxiety is like gardening for your mind. First, recognize that anxiety isn't inherently bad. It's a signal, much like hunger or thirst, telling you something needs attention. The trick is in how you respond to this signal.

Start by shifting your mindset. See anxiety not as a monster lurking in the shadows but as a challenge to be met or even an opportunity for growth. This is the essence of antifragility and resilience. When you feel anxiety bubbling up, ask yourself, "What is this trying to tell me? How can I grow from this?"

Journaling can be a powerful tool here. Spend time writing about your experiences with anxiety, focusing on the lessons learned and the strengths gained from each encounter. This practice can help you find meaning in your stress and align it with your values, transforming it from a source of suffering into a catalyst for achieving your goals.

Remember the principle of "tend and befriend." Helping others can not only reduce your own suffering but also promote feelings of calm and connection through the release of oxytocin. When you're feeling anxious, reach out and support someone else. It's a way of using your stress positively, benefiting both you and those around you.

Lastly, practice self-awareness and mindfulness. Acknowledge your feelings of anxiety without judgment, then consciously choose to transform them into something positive. Whether it's turning fear into excitement or anticipation, or grief into growth, you have the power to change your emotional landscape.

In summary, dealing with anxiety involves a combination of mindset shift, self-reflection, helping others, and transforming negative emotions into positive ones. By practicing these strategies, you can navigate your anxiety more effectively and use it as a tool for personal growth and achievement.

Sources:

[[1](https://bneo.xyz/posts/stress)] The difference lies in your mindset going into it.

[[2](https://bneo.xyz/posts/usfinterview)] 'For I know the plans I have for you,' declares the Lord, 'plans to prosper you and not to harm you, plans to give you a hope and a future. ' — Jeremiah 29:11

[[3](https://bneo.xyz/posts/stress)] there is a fine line between excitement and anxiety

[[4](https://bneo.xyz/posts/emotion-physics)] The power comes in transforming energy, turning signal emotions into positive ones. You can take anxiety and turn it into anticipation, fear into motivation, grief into growth, all of which becomes fuel for growth and action.

[[5](https://bneo.xyz/posts/mentalhealth)] Practical Mental Health TipsFound this 22-minute article that's a mix of personal story, science of mental health, and practical tips.

[[6](https://bneo.xyz/posts/relax)] The lesson here is when you stress out about something, it's likely half of your effort is not effort, it's unnecessary stress that made you feel like you're at your best.



In [39]:
ask("how to focus?")

Question:how to focus?

Answer:
To focus, start by removing distractions. This means turning your phone to grayscale, using focus mode, and batching notifications. If distractions are internal, find what motivates you. Intrinsic motivation, where the work itself is rewarding, can lead to a state of flow and deep focus. If that's lacking, use extrinsic rewards to make task completion satisfying.

On low-focus days, ask yourself if you've slept well, eaten healthily, or exercised enough. Also, consider if the work is too challenging or not aligned with your interests. Adjust your environment and tasks based on these reflections.

Use practical tools like setting specific work hours, taking short breaks, and not engaging in cognitive tasks during those breaks. Music can also help, with energetic tunes for sleepiness and subdued tracks for intense work. Let your mind wander at times instead of constantly seeking stimulation.

Visual techniques like focusing on a specific target can increase alertness. Regularly visualize both positive outcomes and the consequences of failure to maintain motivation. Finally, reward yourself randomly for completing milestones to enhance motivation through dopamine release.

Sources:

[[1](https://bneo.xyz/posts/focus)] Other thoughts:

[[2](https://bneo.xyz/posts/focus)] Tools for Keeping FocusSome things I do to help keep focus on my Macbook and iPhone, inspired by this LessWrong post by Ben Kuhn

[[3](https://bneo.xyz/posts/stillness)] on breaks

[[4](https://bneo.xyz/posts/lowfocus)] Dealing With Low FocusI had a lot of low-focus days the past three months, particularly this month.

[[5](https://bneo.xyz/posts/fujifilm)] You want an image where the brightest points are not almost-but-not-quite 100% white, and the darkest points are almost-but-not-quite 100% black.

[[6](https://bneo.xyz/posts/motivation)] quantify number of hours per week and day for goal



In [40]:
ask("what is success?")

Question:what is success?

Answer:
Success, as distilled from the various insights and reflections provided, isn't merely about achieving career milestones or accumulating accolades. It's a complex blend of perseverance, wisdom, and the intrinsic joy found in the journey of doing what you love. Success involves understanding and overcoming failures, not by dismissing them but by learning from them and moving forward. It's about recognizing that true fulfillment doesn't come from external validation or societal measures of achievement but from deep, meaningful work and relationships. Success is also about self-awareness, knowing the long-term consequences of your actions, and making sacrifices that align with your values and goals. In essence, success is a personal journey of continuous effort, learning, and growth, where the ultimate reward is not just the outcome but the wisdom and satisfaction gained along the way.

Sources:

[[1](https://bneo.xyz/posts/success)] SuccessProfessor G on success.

[[2](https://bneo.xyz/posts/naval)] Make interesting things that people want. Shoe your craft, practice your craft, and the right people will eventually find you

[[3](https://bneo.xyz/posts/culturelies)] A Better Way To LiveNighthawks, 1942, Edward Hopper

[[4](https://bneo.xyz/posts/success-mindset)] Mindset of Successful PeopleIf you feel like everything that happens to you is beyond your control, that there is nothing you can do to set yourself up for success, and not even try, you don't have the mindset of a successful person.

[[5](https://bneo.xyz/posts/burnoutsociety)] Today's society is no longer Foucault's disciplinary society, the 21st century is an achievement society

[[6](https://bneo.xyz/posts/backhome)] After distracting myself with two Sherlock Holmes movie series portrayed by R.D.J., and a third of a quarter of Harry Potter The Deathly Hallows, when my plane landed in Malaysia, I felt immense sadness and yearning. The state of my health shoved me into a corner where I had to give up what I fought for for the past 2 years, the internship hunts and the interviews, the constant pressure to always be learning and growing or I'll never be good enough to land a job, the inability to relax and rest because I was an international student, and by default, I had to work 10x harder. Failure to me meant going back home after graduation, it meant I wasn't good enough, that I failed to live up to my potential, that I have to continue to depend on my father who's already 61 years of age to pursue graduate school. Failure is more complicated and harder to define when it's something out of my control. An external bad actor that has a mind of it's own and specializes in making my life miserable. I



In [41]:
ask("what was 2023 like?")

Question:what was 2023 like?

Answer:
2023 was a year of transitions and growth for you. It started with a memorable graduation trip, filled with adventures across various landscapes and experiences that brought joy and gratitude. You had the opportunity to reconnect with friends, participate in AI events and hackathons, and even host your parents, making you feel more adult. Despite the initial plan to move to the Bay Area and continue with Tesla, life took a different turn, leading you to grad school, which you see as another chance to pursue your goals. The summer was a highlight, with your return to California for internships and networking, enriching your confidence and social skills. However, you also faced challenges, such as dealing with GERD and adjusting to a quieter life back in Iowa for your final class. The year was a mix of achievements, learning, and self-discovery, setting the stage for a purposeful gap year before graduate school in 2024.

Sources:

[[1](https://bneo.xyz/posts/2023)] 2023 reviewFireworks in Nagaoka, Kiyoshi Yamashita, 1950

[[2](https://bneo.xyz/posts/2023)] It's time to review my 2023.

[[3](https://bneo.xyz/posts/22)] Summer 23. It was really nice to be back. I got to see old friends and make new ones. I went for a lot of AI events and hackathons, it felt unreal being in the same room with CEOs and YC founders, sitting at the same table with stanford Ph.D. students, and staying up all night at the ferry building in San Francisco for my app to be presented and awarded. I went to my first american party (and another one later on), although it was more like a networking event since everyone were interns. My parents came to visit for 3 weeks and it was nice driving them around to touristy spots, and eat good food. They liked it a lot, especially since I was paying for everything. I felt like an adult for the first time. It was 14 weeks filled with adventure, thrill, and lots of networking. I'm a little more confident in speaking now, although still trying to be less awkward. During this time, I got back on PPIs cause it started to flare up, but the in-house Tesla doctors told me to stop taking them,

[[4](https://bneo.xyz/posts/22)] The highlight of 2022 was my Tesla internship in Summer 2022. A big part of who I am and what I want today stems from the experiences I had and the people I met in the bay area. Even though I didn't like the work I did the most, being in California expanded my horizons. The world felt really small in Ames, IA. In California, I felt like it was what I was missing the whole time. I felt like I belonged. I had dozens of new experiences and made some of my closest friends during that time, albeit I did had to take PPIs the entire time to sustain myself, it made me feel like I could still live a life with GERD.

[[5](https://bneo.xyz/posts/copy)] #work7/23/2023Next:Live an Asymmetric LifePrevious:AGI House Hackathon

[[6](https://bneo.xyz/posts/22)] Leaving California again was hard. My old friends came back full time, and I felt like I was delaying my life. I also had only one more class to graduate, and I was contemplating a lot about coming back to Iowa. But now I'm glad I did. I was really burnt out in California, and never took the time to rest.



In [42]:
ask("how to make friends?")

Question:how to make friends?

Answer:
Making friends starts with paying attention. Opportunities to connect are everywhere, but they're easy to miss if you're not looking. It's about being open to the unexpected, stepping out of your routine, and embracing the moments that bring you together with someone else. When you feel that click, that sense of connection, it's important to act on it. This means having the courage to let things be awkward at first, being vulnerable, and showing willingness to invest energy and thought into the budding friendship.

Taking action is crucial. Friendship requires effort, like showing up for someone even when it's not convenient. It's about putting in the work to maintain the relationship, which should ultimately bring joy rather than feel like a chore. Establishing rituals can help, such as regular meet-ups or activities that become a part of your routine. This could be anything from book clubs to monthly hikes, making it easier to maintain these connections by simply showing up.

Imagination plays a role too. Be open to making new friends while cherishing the old ones. True friends are honest and push you towards growth, offering support without keeping score. Remember, friendships evolve over time, but the core values that founded them can endure and deepen.

Lastly, it's about choosing friends with care, recognizing that true friendship is based on trust, wisdom, and goodness. Friends should challenge you to be better and support you in becoming the best version of yourself. In today's world, where we're often distracted by screens and superficial connections, making a conscious effort to engage in meaningful friendships is more important than ever. It's about being less available for the distractions and more present for the people who matter.

Sources:

[[1](https://bneo.xyz/posts/proust)] Be a Good Friend

[[2](https://bneo.xyz/posts/friendshipforces)] Attention

[[3](https://bneo.xyz/posts/truefriendship)] make new friends, keep the old: always be open to new friendships, including younger people. no one is a sweeter friend than someone who's been with you from the stat, cherish and maintain them

[[4](https://bneo.xyz/posts/truefriendship)] there are different types of friends: be they business associates, neighbors, or acquaintances, the rare friends we bind ourselves to on a much deeper level are the ones that deeply change our lives, just as we change theirs.

[[5](https://bneo.xyz/posts/play-streets)] Make effort with friends, new and old, and ask for the help and support we need, and providing it in return.

[[6](https://bneo.xyz/posts/play-streets)] We're so burned out from our screen-based, data-heavy lives that we no longer have time or energy for the small, mundane, place-based friendships that has nourished and sustained us for centuries.



In [55]:
ask("qualities of a good friendship")

Question:qualities of a good friendship

Answer:
Qualities of a good friendship, distilled from the essence of the context provided, are multifaceted yet simple at their core. A good friendship is built on honesty; friends tell each other what they need to hear, not necessarily what they want to hear. This honesty is a cornerstone of trust, another indispensable quality. Trust allows friends to share vulnerabilities and confidential information without fear of betrayal.

Loyalty is the foundation that prevents friendships from crumbling under the weight of challenges and changes. It's the assurance that, despite the inevitable evolution of individual lives, the friendship will not only endure but also deepen. Loyalty in friendship means standing by each other, fighting for each other, and pushing each other towards potential, even when it involves uncomfortable truths.

Kindness and the ability to celebrate or console at the right times are qualities that add warmth and depth to a friendship. These actions show that a friend is genuinely invested in your happiness and well-being. Furthermore, a good friendship is characterized by a mutual willingness to help and support each other, whether in times of need or in pursuit of potential.

Lastly, a true friendship is self-rewarding. The ultimate reward of friendship is the friendship itself, not any practical advantages that might come from it. This means that in a true friendship, actions are not motivated by a desire for personal gain but by the value placed on the friendship and the well-being of the friend.

In summary, the qualities of a good friendship include honesty, trust, loyalty, kindness, mutual support, and the intrinsic value placed on the relationship itself. These qualities ensure that the friendship is not only enduring but also enriching for both parties involved.

Sources:

[[1](https://bneo.xyz/posts/proust)] Be a Good Friend

[[2](https://bneo.xyz/posts/truefriendship)] make new friends, keep the old: always be open to new friendships, including younger people. no one is a sweeter friend than someone who's been with you from the stat, cherish and maintain them

[[3](https://bneo.xyz/posts/truefriendship)] there are different types of friends: be they business associates, neighbors, or acquaintances, the rare friends we bind ourselves to on a much deeper level are the ones that deeply change our lives, just as we change theirs.

[[4](https://bneo.xyz/posts/truefriendship)] friendships change over time: all of us change with time, but core values and qualities that formed the friendship survives the test of time. Like fine wine, the best of friendships improve with age.

[[5](https://bneo.xyz/posts/truefriendship)] on loyalty

[[6](https://bneo.xyz/posts/friend)] What is a Friend?



In [43]:
ask("how to get rich?")

Question:how to get rich?

Answer:
To get rich, focus on building and selling. Most effort is wasted, so concentrate on the 1% that isn't. Learn foundational skills and leverage your curiosity and passion to find work that feels like play to you. Wealth comes from using your mind, not trading your time. Understand the three types of leverage: labor, capital, and permissionless leverage like code and media. Make critical decisions about where you live, who you're with, and what you do. Avoid the competition trap by doing what you love authentically, and find a way to serve society with it. Apply leverage, take ownership, and scale up. View creating a business as an art form, akin to play. Build your character and reputation to seize opportunities. Make and share interesting things, and the right people will notice. Choose work that matches your skills and interests, take risks, and work hard without burning out. Stay curious, communicate well, manage effectively, understand finance, and experience different cultures. Finally, set realistic expectations for happiness and understand that achieving the good life requires effort, rethinking desires, and focusing on meaningful relationships.

Sources:

[[1](https://bneo.xyz/posts/naval)] The Almanack of Naval RavikantI finished reading this book in three separate visits to a bookstore while my mother was shopping or getting a haircut.

[[2](https://bneo.xyz/posts/naval)] 1) where you live, 2) who you’re with, and 3) what you do

[[3](https://bneo.xyz/posts/greatwork)] Choose work you have a natural aptitude for and a deep interest in.

[[4](https://bneo.xyz/posts/pmarca-career)] 1) communication: become good public speakers. you'll have a hard time changing the world if you can't communicate well.

[[5](https://bneo.xyz/posts/gradadvice)] Advice for New College GradsSome advice for new college grads by Chris from 2006.

[[6](https://bneo.xyz/posts/gradadvice)] "the good life is a life inspired by love and guided by knowledge" - Bertrand Russell



In [45]:
ask("Advice for data scientists")

Question:Advice for data scientists

Answer:
For data scientists, the journey is both challenging and rewarding. Start by honing your coding skills; platforms like LeetCode can be invaluable for sharpening your problem-solving abilities. Don't overlook the importance of a solid foundation in machine learning principles. Understanding the math behind the algorithms and being able to navigate the trade-offs between different models will set you apart in interviews.

Developing a clear research agenda can guide your projects and make your work more impactful. Remember, research involves a lot of perseverance and sometimes monotonous work, but it's crucial for building deep insights. Communication is key; being able to articulate your findings and blockers clearly will make collaboration with others smoother and more productive.

Experimentation is at the heart of data science. Start with simple models, iterate rapidly, and don't be afraid to explore various approaches. As you refine your model, scaling up becomes the next step, involving more data and more complex architectures. Throughout this process, keep learning from research papers and incorporate external data sources to enhance your models.

Lastly, career growth in data science isn't just about technical skills. Engage with different departments within your company to gain a broader perspective. Try different types of programming to discover what you enjoy most. And always, focus on creating value, both in your projects and in your interactions with others. This holistic approach will not only make you a better data scientist but also a valued team member and innovator in your field.

Sources:

[[1](https://bneo.xyz/posts/ai-jobs)] Actionable Tips

[[2](https://bneo.xyz/posts/research)] The reality of research is a lot of hard work and drudgery, and it's an important part of the research process. It can help you build intuition about the research.

[[3](https://bneo.xyz/posts/ml-guide)] 2. Experiment and research

[[4](https://bneo.xyz/posts/career-advice)] More career advice:

[[5](https://bneo.xyz/posts/advice-programmers)] Talk to people in other parts of the company: support, sales, finance, etc. shadow them if you have the time.

[[6](https://bneo.xyz/posts/saas)] Advice by SaaS Startup FoundersSome advice that Saas startup founders would give to their younger selves.



In [49]:
ask("How to get into AI?")

Question:How to get into AI?

Answer:
To get into AI, start by building a strong foundation in mathematics, especially in calculus and linear algebra, as these are crucial for understanding how AI algorithms work. Resources like "Essence of calculus" and "Essence of linear algebra" are great starting points. Next, dive into machine learning and neural networks through courses like "Machine Learning | Coursera" and "Neural Networks: Zero to Hero by Andrej Karpathy." 

Learning a programming language well-suited for AI, such as Python, and frameworks like PyTorch, will be essential. The "Official PyTorch Tutorials" and tutorials by Aladdin Persson can help you get hands-on experience. Deep learning, transformers, and natural language processing (NLP) are advanced topics you should explore through resources like fast.ai, "Transformers from Scratch," and the "Natural Language Processing Specialization by deeplearning.ai."

Engage with the community and real-world projects through hackathons, like those offered by lablab.ai, and participate in Kaggle competitions to apply what you've learned in a competitive environment. Lastly, always seek feedback, as it's crucial for growth in this fast-paced field. Remember, the key to success in AI is continuous learning and application of knowledge.

Sources:

[[1](https://bneo.xyz/posts/ai-engineer)] AI Engineer curriculumA list of great resources for breaking into AI Engineering.

[[2](https://bneo.xyz/posts/stefan)] What are some common mistakes candidates make?

[[3](https://bneo.xyz/posts/ai-jobs)] High yield: Candidates from prestigious schools can experience high response rates to job applications, allowing more time for reflective interview preparation rather than application quantity.

[[4](https://bneo.xyz/posts/ai-jobs)] Resources

[[5](https://bneo.xyz/posts/research)] Most Important Traits for AI Research

[[6](https://bneo.xyz/posts/310324)] coding



In [47]:
ask("What is climate change?")

Question:What is climate change?

Answer:
Climate change is a big shift in our planet's weather and temperature patterns. It's happening because humans are doing things that increase the amount of greenhouse gases (GHGs) in the atmosphere. These gases, like carbon dioxide (CO2) and methane, trap heat from the sun, making the Earth warmer. This warming is causing problems like melting ice caps, rising sea levels, and extreme weather events. To fix it, we need to stop adding new CO2 into the atmosphere and start taking out what's already there. We also have to deal with the impacts of the warming that's already happening. It's a big challenge, but understanding it is the first step to solving it.

Sources:

[[1](https://bneo.xyz/posts/ghg)] Tracking Carbon EmissionCoalbrookdale by Night, 1801, Philip James de Loutherbourg

[[2](https://bneo.xyz/posts/5qsclimate)] 5 Questions to Ask in ClimateI'm on Chapter 3 of Bill Gates' book on climate.

[[3](https://bneo.xyz/posts/climate)] Resources for Getting Into ClimateEdvard Munch, The Sun, 1911

[[4](https://bneo.xyz/posts/so2)] Stop putting new CO2

[[5](https://bneo.xyz/posts/so2)] GeoengineeringIt's scary times here on Earth. The North Atlantic Sea Surface temperature is higher than ever.

[[6](https://bneo.xyz/posts/climate)] communities



In [48]:
ask("What foods should I eat more to stay healthy?")

Question:What foods should I eat more to stay healthy?

Answer:
To stay healthy, focus on eating more foods that support your gut health and overall well-being. This includes incorporating more probiotics, prebiotics, fiber, polyphenols, and fermented foods into your diet. Probiotics are found in fermented foods like yogurt, kefir, sauerkraut, and kimchi, which help maintain a healthy balance of gut bacteria. Prebiotic foods, such as whole grains, bananas, greens, onions, garlic, soybeans, and artichokes, provide the necessary fuel for your gut bacteria to thrive. Eating a diet rich in dietary fiber from fruits, vegetables, nuts, legumes, and whole grains not only fuels your gut bacteria but also promotes the production of beneficial short-chain fatty acids. Polyphenols, found in foods like berries, nuts, seeds, olive oil, and green tea, have antioxidant properties that can benefit your gut health and overall health. By focusing on these foods, you can support your digestive system, immune system, mental health, weight management, blood sugar levels, and reduce inflammation.

Sources:

[[1](https://bneo.xyz/posts/gut)] Eat more: Probiotics, prebiotics, fiber, polyphenols and fermented foods

[[2](https://bneo.xyz/posts/22)] The second biggest change was my diet. I slowly realized the amount of limitations placed on my diet as days turned into weeks, and into months. My two favorite things at that time: green tea and chocolate was out of my life from that day. Anything with pepper is off limits. I could no longer drink ice cold carbonated drinks on a hot day. I couldn't take sips of alcohol or eat mandarin oranges during Chinese New Years. No more fried chicken from Texas Chicken. No eating the crispy skin on roasted pork. And so much more. At that time, I lived on a diet of thin-noodle soup and fish paste which my mum made most days at home. It became my favorite food.

[[3](https://bneo.xyz/posts/gut)] They help with food digestion.

[[4](https://bneo.xyz/posts/meats)] less land and water

[[5](https://bneo.xyz/posts/meats)] same nutritional qualities as red meat

[[6](https://bneo.xyz/posts/surgeryday2)] I only started eating at 2 p.m. My mom brought blended porridge. It was bright orange because of the pumpkins.



In [54]:
ask("How are you doing?")

Question:How are you doing?

Answer:
I don't know.

Sources:

[[1](https://bneo.xyz/posts/good-day)] Good DayDon't ask whether today will be a good day.

[[2](https://bneo.xyz/posts/goodconvos)] the worst questions

[[3](https://bneo.xyz/posts/work)] Another day at workYou get into the office on a Monday morning, your colleague besides you asks how your weekend went.

[[4](https://bneo.xyz/posts/goodconvos)] Questions about the positive sides of life

[[5](https://bneo.xyz/posts/surgeryday)] The surgeon came in and told me everything went very well. He assured me that what I was feeling was normal, and that he'll see me the next morning.

[[6](https://bneo.xyz/posts/proust)] Be a Good Friend



In [44]:
ask("how to kill a person?")

Question:how to kill a person?

Answer:
I don't know.

Sources:

[[1](https://bneo.xyz/posts/death)] O, Death!

[[2](https://bneo.xyz/posts/36)] If you die this evening without the opportunity to communicate with anyone, what would you most regret not having told someone? Why haven’t you told them yet?

[[3](https://bneo.xyz/posts/hemmingway)] Towards the end of his life, he was admitted to a psychiatric hospital.

[[4](https://bneo.xyz/posts/death)] It was depressing he felt so isolated in his deathbed, he was surrounded by the most shallow of people, even his wife, and just wanted someone to comfort and cry with him.

[[5](https://bneo.xyz/posts/death)] A life poorly lived only amplifies the horrors of death or create them entirely. So I believe the answer in death is not found in what happens after or during or even why we all must die, but instead in how we live.

[[6](https://bneo.xyz/posts/happy-life)] "Don't ever confuse the two, your life and your work"



In [46]:
ask("Is heaven real?")

Question:Is heaven real?

Answer:
I don't know.

Sources:

[[1](https://bneo.xyz/posts/proust)] Open your Eyes

[[2](https://bneo.xyz/posts/psalms23)] He longs to give us peace, rest, and takes us to where water is pure, and almost motionless so we refresh ourselves in Him.

[[3](https://bneo.xyz/posts/life-proverb)] Pastor Kieth shared this at the service at Renewal today for his sermon about life.

[[4](https://bneo.xyz/posts/psalms23)] This is a widely recognized Psalm that offers us peace and reminds us that God is guiding us along His path, and he has things in control.

[[5](https://bneo.xyz/posts/culturelies)] your heart years for fusion with another

[[6](https://bneo.xyz/posts/littlejoys)] Just try it once — a tree, or at least a considerable section of sky, is to be seen anywhere. It does not even have to be blue sky; in some way or another the light of the sun always makes itself felt. Accustom yourself every morning to look for a moment at the sky and suddenly you will be aware of the air around you, the scent of morning freshness that is bestowed on you between sleep and labor. You will find every day that the gable of every house has its own particular look, its own special lighting. Pay it some heed if you will have for the rest of the day a remnant of satisfaction and a touch of coexistence with nature. Gradually and without effort the eye trains itself to transmit many small delights, to contemplate nature and the city streets, to appreciate the inexhaustible fun of daily life. From there on to the fully trained artistic eye is the smaller half of the journey; the principal thing is the beginning, the opening of the eyes.



In [ ]:
# cleanup
vectorstore.delete_collection()